In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('sql_conns.db')

In [29]:
orders = pd.read_csv('desktop/order.csv', sep=';')
product = pd.read_csv('desktop/product.csv', sep=';')

In [150]:
orders.head()

,order_id,accepted_at,product_id,quantity,regular_price,price,cost_price
0,1516729899,2022-01-10 16:37:37,10163,1,329,329,254
1,1518335746,2022-01-16 12:22:02,10163,1,329,329,229
2,1518354200,2022-01-15 17:03:41,10163,1,329,329,229
3,1516838503,2022-01-11 12:31:53,10163,1,329,329,254
4,1516739758,2022-01-10 19:33:35,10163,1,329,329,254


In [149]:
print(orders['accepted_at'].min())
print(orders['accepted_at'].max())

2022-01-10 08:08:44
2022-01-16 22:59:00


In [80]:
orders.to_sql('orders', conn, index=False, if_exists='replace')
product.to_sql('product', conn, index=False, if_exists='replace')

3573

### Самое высокое проникновение в трафик

Не очень поняла, что здесь имеется ввиду. Но имея данные только о продажах, посмотрю категорию лидер по заказам, рублям и количеству товаров.

In [163]:
sql = '''
select 
*, 
rank() over(order by orders desc) as rnk_orders,
rank() over(order by qnt desc) as rnk_qnt,
rank() over(order by sales desc) as rnk_sales
from
(select p.level1, 
       count(distinct o.order_id) as orders,
       sum(o.quantity) as qnt, 
       sum(o.price*o.quantity) as sales 
from orders as o
join product as p on p.product_id = o.product_id
group by p.level1)
'''

In [164]:
dt = pd.read_sql(sql, conn)
dt

,level1,orders,qnt,sales,rnk_orders,rnk_qnt,rnk_sales
0,Безалкогольные напитки,1598,3767,329748,1,1,1
1,Молочная продукция,1497,3766,308628,2,2,3
2,Хлеб и хлебобулочные изделия,1089,1637,111682,3,7,12
3,Свежие овощи,985,2019,273249,4,3,5
4,Замороженная продукция,933,1868,282128,5,5,4
5,Кулинария,890,1907,309166,6,4,2
6,Бакалея,867,1753,168323,7,6,8
7,Свежие фрукты,775,1220,200103,8,9,6
8,Снэки,728,1439,150740,9,8,9
9,Сыры,643,859,149556,10,14,10


Явным лидером является категория Безалкогольные напитки. Товары данной категории лежали в наибольшем количестве заказов. Также продажа этих товаров принесла наибольшую прибыль.

### Средний чек 13.01

In [154]:
sql1 = '''
select sum(price)/count(order_id) as avg_check
from
(select order_id, sum(price*quantity) as price 
from orders
where date(accepted_at) = '2022-01-13'
group by order_id)
'''

In [155]:
dt1 = pd.read_sql(sql1, conn)
dt1

,avg_check
0,915


Средний чек 13 января равен 915 рублей.

### Доля промо в категории Сыры

In [143]:
sql2 = '''
select *,
       sum(qnt) over() as total_qnt,
       sum(sales) over() as total_sales,
       round(cast(qnt as float)/sum(qnt) over() * 100, 2) as share_qnt,
       round(cast(sales as float)/sum(sales) over() * 100, 2) as share_sales
       
from
(select p.level1, 
       case when o.price < o.regular_price then 'promo'
       else 'regular'
       end as sale_type,
       sum(o.quantity) as qnt,
       sum(o.quantity) * sum(o.price)  as sales
from orders as o
join product as p on p.product_id = o.product_id
where p.level1 = 'Сыры'
group by p.level1, sale_type)
'''

In [144]:
dt4 = pd.read_sql(sql2, conn)
dt4

,level1,sale_type,qnt,sales,total_qnt,total_sales,share_qnt,share_sales
0,Сыры,promo,260,10175620,859,71826497,30.27,14.17
1,Сыры,regular,599,61650877,859,71826497,69.73,85.83


- 30% проданных товаров в категории Сыры были Промо товары 
- 14% продаж в категории Сыры пришлось на Промо товары

### Потребление по категории Птица

In [115]:
sql3 = '''
select p.level1, sum(o.quantity) as qnt, sum(o.price*o.quantity) as sales 
from orders as o
join product as p on p.product_id = o.product_id
where p.level1 = 'Птица'
group by p.level1 
'''

In [116]:
dt2 = pd.read_sql(sql3, conn)
dt2

,level1,qnt,sales
0,Птица,541,122200


По категории Птица было куплено 541 товаров, на сумму 122 200 рублей.

### Маржа по категории Молочная продукция

In [117]:
sql4 = '''
select name, mrg, round(cast(mrg as float)/cast(sales as float)*100,2) as mrg2
from
(select p.level1 as name, 
       sum(o.price*o.quantity) - sum(o.cost_price*o.quantity) as mrg,  
       sum(o.price*o.quantity) as sales
from orders as o
join product as p on p.product_id = o.product_id
where p.level1 = 'Молочная продукция'
group by p.level1 )
'''

In [118]:
dt3 = pd.read_sql(sql4, conn)
dt3

,name,mrg,mrg2
0,Молочная продукция,106243,34.42


Маржа по категории Молочная продукция составляет 106 234 рублей или 34%